In [2]:
import os
import random
import pandas as pd
import numpy as np
import re
import json

import matplotlib.pyplot as plt
import seaborn as sns

from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import (
    get_raw_data, 
    parse_filename, 
    expand_acc_data, 
    expand_ppg_data, 
    merge_sensor_data,
    expand_sensor_data,
    process_and_merge_sensor_data
)
from script.wear_detection_dll import *

# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

In [18]:
%matplotlib qt
# %matplotlib inline
is_ring = False




# directory = r'G:\PyScript\rawdata\Wear\BUG'
# file_name = "HeartRate_Static_xf_20260122_162945_afc2.rawdata"

directory = r'G:\PyScript\rawdata\Wear'
file_name = "HeartRate_Daylife_Jeven_20260123_145805_0af6.rawdata"

is_ring = True
file_path = os.path.join(directory, file_name)

real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
info =  parse_filename(file_name)
mac = info.get('mac','0000').lower()

print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

result = get_raw_data(real_path,is_ring=is_ring)

save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)


data_type_dict = result.get('data_types', {})

base_info_df = result.get('base_info_df', pd.DataFrame(columns=['unix_timestamp']))
wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_3697_data_df = result.get('TYHX_3697_data_df', pd.DataFrame(columns=['unix_timestamp']))

ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
acc_df =  result.get('acc_df', pd.DataFrame(columns=['unix_timestamp']))


sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 


print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")

(combined_ppg_df, 
wear_info_df,
expanded_acc_df, 
expanded_ppg_g_df, 
expanded_ppg_ir_df, 
expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 

expanded_ppg_r_ir_df = expand_ppg_data(ppg_r_ir_df, 'ppg_data')



# tyhx_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_data.csv", index=False, encoding='utf-8')
# tyhx_agc_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_agc_data.csv", index=False, encoding='utf-8')
# combined_ppg_df.to_csv(f"{save_dir}/{file_name_without_ext}_combined_ppg.csv", index=False, encoding='utf-8')
# expanded_ppg_g_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_g.csv", index=False, encoding='utf-8')
# expanded_ppg_r_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r.csv", index=False, encoding='utf-8')
# expanded_ppg_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_ir.csv", index=False, encoding='utf-8')
# expanded_ppg_r_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r_ir.csv", index=False, encoding='utf-8')
# acc_df.to_csv(f"{save_dir}/{file_name_without_ext}_acc_data.csv", index=False, encoding='utf-8')


real_path: G:\PyScript\rawdata\Wear\HeartRate_Daylife_Jeven_20260123_145805_0af6.rawdata G:\PyScript\rawdata\Wear HeartRate_Daylife_Jeven_20260123_145805_0af6.rawdata  0af6 

文件 G:\PyScript\rawdata\Wear\HeartRate_Daylife_Jeven_20260123_145805_0af6.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 3577 
15  RAW_PPG_GREEN         原始PPG绿光                count: 1098 
17  RAW_PPG_IR            原始PPG红外                count: 2501 
18  RAW_ACC               原始加速度                  count: 7587 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 2172 
33  AMBIENT_INFO          环境信息                   count: 1098 
67  SLEEP_INFO            睡眠信息                   count: 3576 
68  HRV_INFO              心率变异性信息                count: 656 
71  WEAR_INFO             佩戴信息                   count: 3599 
78  HEART_ACC_INFO        心率加速度信息                count: 1098 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 3599 
91  GREEN_PSP_INFO        绿光PSP信息                

In [5]:


def _smooth_packet_counts_logic(counts_array, tolerance):
    """
    (内部算法) 寻找堆积的包(>1),尝试将其移动到附近的空位(0)。
    实现受限的负载均衡。
    """
    smoothed = counts_array.copy()
    n = len(smoothed)
    
    for i in range(n):
        while smoothed[i] > 1:
            found_gap = False
            best_target = -1
            blocked_left = False
            blocked_right = False
            
            for dist in range(1, tolerance + 1):
                # Check Left
                left_idx = i - dist
                if not blocked_left and left_idx >= 0:
                    val = smoothed[left_idx]
                    if val == 0:
                        best_target = left_idx
                        found_gap = True
                        break 
                    elif val > 1:
                        blocked_left = True
                
                # Check Right
                right_idx = i + dist
                if not blocked_right and right_idx < n:
                    val = smoothed[right_idx]
                    if val == 0:
                        best_target = right_idx
                        found_gap = True
                        break
                    elif val > 1:
                        blocked_right = True
                
                if (blocked_left or left_idx < 0) and (blocked_right or right_idx >= n):
                    break
            
            if found_gap:
                smoothed[i] -= 1
                smoothed[best_target] += 1
            else:
                break
    return smoothed

def assign_segments(df: pd.DataFrame, time_col='unix_timestamp', tolerance=4) -> pd.DataFrame:
    """
    第一步：数据分段
    根据时间戳的跳变(Gap)和回退(Rollback)标记 seg_id。
    """
    df = df.copy()
    # 确保是物理到达顺序
    df = df.sort_index()
    
    # 转换为数值型,防止字符串干扰计算
    ts_series = df[time_col].astype(float) # 允许 NaN 存在
    
    # 计算差分
    diffs = ts_series.diff()
    
    # 标记断点：间距 > tolerance 或 负值 (回退)
    is_break = (diffs > tolerance) | (diffs < 0)
    is_break = is_break.fillna(False)
    
    # 生成段 ID
    df['seg_id'] = is_break.cumsum()
    return df

def generate_smoothed_skeleton(segmented_df: pd.DataFrame, time_col='unix_timestamp', tolerance=4) -> pd.DataFrame:
    """
    第二步：生成宏观骨架并平滑
    1. 生成完美的 1Hz 网格。
    2. 统计原始包量。
    3. 执行平滑算法。
    """
    # 1. 计算每一段的范围 (Min, Max)
    # dropna 确保不处理无效时间戳
    valid_data = segmented_df.dropna(subset=[time_col])
    seg_ranges = valid_data.groupby('seg_id')[time_col].agg(['min', 'max'])
    
    # 2. 生成标准 1Hz 序列
    # range(min, max+1) 需要整数
    seg_ranges['uniform_unix'] = [
        range(int(row['min']), int(row['max']) + 1) 
        for _, row in seg_ranges.iterrows()
    ]
    
    # 炸开生成骨架 DF
    skeleton_df = seg_ranges.explode('uniform_unix')[['uniform_unix']].reset_index()
    skeleton_df['uniform_unix'] = skeleton_df['uniform_unix'].astype(int)
    
    # 3. 统计原始数据分布
    # 按 seg_id 和 timestamp 联合分组
    raw_counts = valid_data.groupby(['seg_id', time_col]).size().reset_index(name='raw_count')
    
    # 4. 合并数据
    final_df = pd.merge(
        skeleton_df, 
        raw_counts, 
        left_on=['seg_id', 'uniform_unix'], 
        right_on=['seg_id', time_col], 
        how='left'
    )
    
    # 5. 填充空缺 (Gap = 0)
    final_df['packet_count'] = final_df['raw_count'].fillna(0).astype(int)
    
    # 6. 执行核心平滑算法
    final_df['smoothed_count'] = _smooth_packet_counts_logic(
        final_df['packet_count'].values, 
        tolerance=tolerance
    )
    
    # 只保留关键列
    return final_df[['seg_id', 'uniform_unix', 'packet_count', 'smoothed_count']]

def apply_correction_to_rows(segmented_df: pd.DataFrame, skeleton_df: pd.DataFrame) -> pd.DataFrame:
    """
    第三步：微观映射
    将宏观的 smoothed_count 展开并映射回原始数据行。
    """
    result_df = segmented_df.copy()
    result_df['final_corrected_unix'] = result_df['unix_timestamp'] # 默认值
    
    # 获取涉及的所有段
    seg_ids = skeleton_df['seg_id'].unique()
    
    for seg_id in seg_ids:
        # 获取宏观骨架数据
        mask_skel = skeleton_df['seg_id'] == seg_id
        segment_skel = skeleton_df[mask_skel]
        
        # 获取微观原始数据索引
        mask_orig = result_df['seg_id'] == seg_id
        segment_orig_indices = result_df[mask_orig].index
        
        # 展开时间轴 (Repeat)
        # 例如: Time=100, Count=2 -> [100, 100]
        expanded_timestamps = np.repeat(
            segment_skel['uniform_unix'].values, 
            segment_skel['smoothed_count'].values
        )
        
        # 长度校验与赋值
        n_orig = len(segment_orig_indices)
        n_new = len(expanded_timestamps)
        
        if n_orig == n_new:
            result_df.loc[segment_orig_indices, 'final_corrected_unix'] = expanded_timestamps
        else:
            # 容错处理：取交集长度
            min_len = min(n_orig, n_new)
            print(f"Warning: Segment {seg_id} length mismatch. Orig: {n_orig}, New: {n_new}. Truncating.")
            result_df.loc[segment_orig_indices[:min_len], 'final_corrected_unix'] = expanded_timestamps[:min_len]
            
    return result_df

In [6]:
def process_timestamp_correction(df: pd.DataFrame, tolerance: int = 4) -> pd.DataFrame:
    """
    主管道函数：执行完整的时间戳清洗与平滑流程。
    """
    print("1. Starting Segmentation...")
    df_segmented = assign_segments(df, tolerance=tolerance)
    
    print("2. Generating Smoothed Skeleton...")
    df_skeleton = generate_smoothed_skeleton(df_segmented, tolerance=tolerance)
    
    print("3. Applying Corrections to Rows...")
    df_final = apply_correction_to_rows(df_segmented, df_skeleton)
    
    print("Done.")
    return df_final

In [21]:
corrected_df = process_timestamp_correction(base_info_df, tolerance=4)

# 查看修正前后的对比
cols = ['unix_timestamp', 'final_corrected_unix', 'seg_id']
print(corrected_df[cols].head(20))

1. Starting Segmentation...
2. Generating Smoothed Skeleton...
3. Applying Corrections to Rows...
Done.
    unix_timestamp  final_corrected_unix  seg_id
0       1769180286            1769180286       0
1       1769180287            1769180287       0
2       1769180288            1769180288       0
3       1769180289            1769180289       0
4       1769180290            1769180290       0
5       1769180291            1769180291       0
6       1769180292            1769180292       0
7       1769180293            1769180293       0
8       1769180294            1769180294       0
9       1769180295            1769180295       0
10      1769180296            1769180296       0
11      1769180297            1769180297       0
12      1769180298            1769180298       0
13      1769180299            1769180299       0
14      1769180300            1769180300       0
15      1769180301            1769180301       0
16      1769180302            1769180302       0
17      176918

In [22]:
df = corrected_df[['unix_timestamp','final_corrected_unix']].copy()
df['diff'] = df['unix_timestamp'] - df['final_corrected_unix']

In [ ]:
df.to_clipboard()

In [11]:

def detect_jitter_patterns(df, time_col='unix_timestamp'):
    """
    输入: 
        df: 包含时间戳数据的 DataFrame
        time_col: 时间戳列名
    
    输出:
        burst_df: 统计多包堆积的 DataFrame
        gap_df: 统计时间空缺的 DataFrame
    """
    # 确保数据按行号(Index)排序,模拟接收顺序
    # 如果你的行号是单独一列,请先 df.set_index('row_id'),或者修改这里
    df = df.sort_index()
    
    # ==========================
    # 1. 检测多包堆积 (Burst)
    # ==========================
    # 按时间戳分组,统计每秒的包数
    group_counts = df.groupby(time_col).size()
    
    # 筛选出包数 > 1 的时间点
    burst_ts = group_counts[group_counts > 1].index
    
    burst_list = []
    for ts in burst_ts:
        # 获取该时间戳对应的所有行号
        rows = df[df[time_col] == ts].index.tolist()
        burst_list.append({
            'unix_timestamp': ts,
            'count': len(rows),
            'row_indices': rows  # 记录涉及的行号
        })
    
    burst_df = pd.DataFrame(burst_list)
    # 调整列顺序以便查看
    if not burst_df.empty:
        burst_df = burst_df[['unix_timestamp', 'count', 'row_indices']]

    # ==========================
    # 2. 检测时间缺失 (Gap)
    # ==========================
    # 计算当前行时间戳与上一行时间戳的差值
    # diff = current_ts - prev_ts
    time_diff = df[time_col].diff()
    
    # 正常情况 diff 应该 <= 1 (0是堆积,1是正常,>1是缺失)
    # 筛选出差值 > 1 的行,这代表它和上一行之间有断档
    gap_indices = time_diff[time_diff > 1].index
    
    gap_list = []
    for idx in gap_indices:
        # 当前行(Gap结束后的第一行)
        curr_row = df.loc[idx]
        curr_ts = curr_row[time_col]
        
        # 上一行(Gap开始前的最后一行)
        # 注意：这里假设 DataFrame 的 index 是连续整数。如果不是,需要用 iloc
        # 为了通用性,我们用 shift 获取上一行的值
        prev_ts = df.loc[idx, time_col] - time_diff.loc[idx] # 反推上一行时间戳
        
        # 缺失的长度 = 差值 - 1
        # 比如：上一行是 100,当前行是 102。diff=2。缺失的是 101,长度为 1。
        missing_len = int(time_diff.loc[idx] - 1)
        
        # 缺失的起始时间戳
        start_missing_ts = int(prev_ts + 1)
        
        gap_list.append({
            'row_pair': (idx - 1, idx), # 在哪两行之间发生的断裂
            'gap_start_unix': start_missing_ts,
            'missing_seconds': missing_len
        })
        
    gap_df = pd.DataFrame(gap_list)

    return burst_df, gap_df

In [12]:
# gyro_df: 陀螺仪数据 DataFrame
# ppg_r_df: 红光 PPG 数据 DataFrame
# ppg_r_ir_df: 红外红光 PPG 数据 DataFrame
# TYHX_data_df: TYHX 数据 DataFrame
# TYHX_agc_data_df: TYHX AGC 数据 DataFrame
# TYHX_3697_data_df: TYHX 3697 数据 DataFrame (戒指设备)
# data_types: 数据类型统计字典

# acc_df: 加速度数据 DataFrame
# ppg_g_df: 绿光 PPG 数据 DataFrame
# ppg_ir_df: 红外光 PPG 数据 DataFrame
# base_info_df: 基础信息 DataFrame
# wear_info_df: 佩戴信息 DataFrame

In [23]:
bursts, gaps = detect_jitter_patterns(acc_df, 'unix_timestamp')

print("--- 1. 多包堆积检测 (Bursts) ---")
print(bursts.to_string(index=False))
print("\n--- 2. 时间空缺检测 (Gaps) ---")
print(gaps .to_string(index=False) if 'gap_df' in locals() else gaps.to_string(index=False))

--- 1. 多包堆积检测 (Bursts) ---
 unix_timestamp  count                                                                                                                                       row_indices
     1769180550      2                                                                                                                                        [264, 265]
     1769180551      6                                                                                                                    [266, 267, 268, 269, 270, 271]
     1769180577      2                                                                                                                                        [289, 290]
     1769180594      4                                                                                                                              [291, 292, 293, 294]
     1769180602      2                                                                                                          

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(20, 16), sharex=True)

# 绘制 PPG Green
if expanded_ppg_g_df is not None and not expanded_ppg_g_df.empty:
    axes[0].plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], 
                 label='PPG Green', color='green', linewidth=1)
    axes[0].set_ylabel('PPG Green Value')
    axes[0].set_title('PPG Green Data')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# 绘制 PPG Red
if expanded_ppg_r_df is not None and not expanded_ppg_r_df.empty:
    axes[1].plot(expanded_ppg_r_df['unix_timestamp'], expanded_ppg_r_df['ppg_value'], 
                 label='PPG Red', color='red', linewidth=1)
    axes[1].set_ylabel('PPG Red Value')
    axes[1].set_title('PPG Red Data')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

# 绘制 PPG R_IR
if expanded_ppg_r_ir_df is not None and not expanded_ppg_r_ir_df.empty:
    axes[2].plot(expanded_ppg_r_ir_df['unix_timestamp'], expanded_ppg_r_ir_df['ppg_value'], 
                 label='PPG R_IR', color='purple', linewidth=1)
    axes[2].set_ylabel('PPG R_IR Value')
    axes[2].set_title('PPG R_IR Data')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)


# 绘制 PPG IR
if expanded_ppg_ir_df is not None and not expanded_ppg_ir_df.empty:
    axes[3].plot(expanded_ppg_ir_df['unix_timestamp'], expanded_ppg_ir_df['ppg_value'], 
                 label='PPG IR', color='pink', linewidth=1)
    axes[3].set_ylabel('PPG IR Value')
    axes[3].set_title('PPG IR Data')
    axes[3].set_xlabel('Unix Timestamp (s)')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
